# Set-up
For faster implementation, please create three empty folders on your Google Drive, named as 'Lease','Processed_pdf', and 'Pdf_text'.

Also, please find your sharepoint username and sharepoint password

In [1]:
# Install Packages
! pip install Office365-REST-Python-Client
! pip install PyPDF2
! pip install pdfminer
! pip install -q transformers
!sudo apt install tesseract-ocr
!pip install pytesseract
! pip install pdf2image
!apt-get install poppler-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 942.9/942.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.0 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140105 sha256=bf3b37e26a5a4ab00ac9c2937aa0acfbfc96093dc885d2c309cfcc978bc54a10
  Stored in directory: /root/.cache/pip/wheels/4e/c1/68/f7bd0a8f514661f76b5cbe3b5f76e0033d79f1296012cbbf72
Successfully built pdfminer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Upgrade packages
!sudo apt-get dist-upgrade

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libcudnn8 libcudnn8-dev libnccl-dev libnccl2
The following packages will be upgraded:
  binutils binutils-common binutils-x86-64-linux-gnu cuda-keyring
  cuda-toolkit-config-common libbinutils libcap2 libctf-nobfd0 libctf0
  libldap-2.5-0 linux-libc-dev
11 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Need to get 4,954 kB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  cuda-keyring 1.1-1 [4,332 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  cuda-toolkit-config-common 12.2.128-1 [16.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libcap2 amd64 1:2.44-1ubuntu0.22.04.1 [18.3 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main a

In [98]:
# Import libraries and modules
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
import os
import pytesseract
from pytesseract import Output
import cv2
from google.colab.patches import cv2_imshow
import re
import pytesseract
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image
import tempfile
from pathlib import Path
import PIL
from PIL import ImageDraw
from pdf2image import convert_from_path, convert_from_bytes
import pandas as pd

# Connect to Sharepoint

In [ ]:
sharepoint_url = "https://davittjonesbouldltd.sharepoint.com/sites/Files"
sharepoint_files = 'Shared Documents'
site_name = 'Files'
username = 'difan.deng@djblaw.co.uk'
password = 'PaperMoon543!'
folder_name = 'Adminwork/Difan Deng/Lease'

In [ ]:
# Build a connection
connection_sharepoint = ClientContext(sharepoint_url).with_credentials(
            UserCredential(
                username,
                password
            )
        )

# Download file from sharepoint

In [ ]:
# Define a function to extract files from folders and download
def get_folders(files,folder):
        # Establish the connection
        connection = connection_sharepoint
        # Get list of folders in the main folder
        target_folder_url = f'{files}/{folder}'
        root_folder = connection.web.get_folder_by_server_relative_url(target_folder_url)
        subfolders = root_folder.folders
        connection.load(subfolders)
        connection.execute_query()
        folder_names = []
        # Get all folders in the 'Lease' folder
        for s_folder in subfolders:
            folder_names.append(s_folder.properties['Name'])
        # Loop over each folder to get the files
        folder_names = [i for i in folder_names if 'cvs' in i.lower()] # This line is spcified for CVS
        for i in range(len(folder_names)):
          folder_url = f'{files}/{folder}/{folder_names[i]}'
          file_source = connection.web.get_folder_by_server_relative_url(folder_url)
          file_list = file_source.files
          connection.load(file_list)
          connection.execute_query()
          file_name = []
          for file in file_list:
            file_name.append(file.properties['Name'])
        # Download the file if file_name is not empty
          for x in range(len(file_name)):
            if (('lease' in file_name[x].lower()) and ('pdf' in file_name[x])): # only download lease file & pdf files
              file_name_x = file_name[x]
              with open(f"/content/drive/MyDrive/Lease/{file_name_x}", 'wb') as output_file: # Specify the output location
                file_url = folder_url + '/' + file_name_x
                file_connection = connection.web.get_file_by_server_relative_url(file_url)
                file_connection.download(output_file)
                connection.execute_query()
                '''
            else: # Remove unnecessary documents to save storage
              file_name_y = file_name[x]
              file_url = folder_url + '/' + file_name_y
              file_connection = connection.web.get_file_by_server_relative_url(file_url)
              file_connection.delete_object()
              connection.execute_query()
            # Do not download non-lease files
            '''
            x+=1
          i+=1

In [ ]:
        connection = connection_sharepoint
        # Get list of folders in the main folder
        target_folder_url = f'{sharepoint_files}/{folder_name}'
        root_folder = connection.web.get_folder_by_server_relative_url(target_folder_url)
        subfolders = root_folder.folders
        connection.load(subfolders)
        connection.execute_query()
        folder_names = []
        # Get all folders in the 'Lease' folder
        for s_folder in subfolders:
            folder_names.append(s_folder.properties['Name'])

In [ ]:
# Get the list of file directories in the sharepoint
file_name = []
for i in range(len(folder_names)):
    folder_url = f'{sharepoint_files}/{folder_name}/{folder_names[i]}'
    file_source = connection.web.get_folder_by_server_relative_url(folder_url)
    file_list = file_source.files
    connection.load(file_list)
    connection.execute_query()
    for file in file_list:
        file_name.append(file.properties['Name'])
    i+=1

In [ ]:
# Download files by file directories
get_folders(sharepoint_files,folder_name)

In [ ]:
# check length of the direcotry
dir = '/content/drive/MyDrive/Lease'
leases_NUm = os.listdir(dir)
len(leases_NUm)

390

# Classify pdf documents into text or image

In [ ]:
# Find all file path for leases in the folder
dir = '/content/drive/MyDrive/Lease' # same as output directory
lease_path = []
for path in os.listdir(dir):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir, path)):
      lease_path.append(dir + '/' + path)

In [ ]:
# Functions to classify the pdfs into either text-based or image-based
pdf_types = []
error_pdf = []
def get_pdf_searchable_pages(fname):
    """ intentifying a digitally created pdf or a scanned pdf"""
    from pdfminer.pdfpage import PDFPage
    searchable_pages = []
    non_searchable_pages = []
    page_num = 2
    try:
      with open(fname, 'rb') as infile:
        for page in PDFPage.get_pages(infile,check_extractable=False):
            page_num += 1
            if 'Font' in page.resources.keys():
                searchable_pages.append(page_num)
            else:
                non_searchable_pages.append(page_num)
      if page_num == len(searchable_pages):
          pdf_types.append(1)
      elif page_num != len(searchable_pages):
          pdf_types.append(0)
      else:
          pdf_types.append("fault")
    except:
      error_pdf.append(fname)
      lease_path.remove(fname)

In [ ]:
# Classify the pdfs
for i in lease_path:
  get_pdf_searchable_pages(i)

In [ ]:
# Split csv leases accordingly to text pdf and image pdf
def find_indices(list_to_search, values):
    indices = []
    for idx, value in enumerate(list_to_search):
        if value == values:
            indices.append(idx)
    return indices
text_pdf_indices = find_indices(pdf_types, 1)
image_pdf_indices = find_indices(pdf_types,0)

In [ ]:
# Image-based filepath
image_pdf = []
for i in image_pdf_indices:
  image_pdf.append(lease_path[i])
# Test-nased filepath
text_pdf = []
for i in text_pdf_indices:
  text_pdf.append(lease_path[i])

In [ ]:
len(lease_path)

92

# PDF to Image and Image preprocessing

In [ ]:
from pdf2image.exceptions import (
PDFInfoNotInstalledError,
PDFPageCountError,
PDFSyntaxError
)

## Convert PDFs to images

In [ ]:
# Construct a function that transforms image-based pdfs to images
import shutil
shutil.rmtree
def pdf_to_image(file_path):
  # Find all filepath that contains an error
  file_path = f"{file_path}"
  error_file = []
  missing_file = []
  check_file = os.path.exists(file_path)
  # write file
  if check_file==True:
    try:
      name = file_path.split('/')[-1][:-4] # Get the name of leases
      folder_name = '/content/drive/MyDrive/Processed_pdf/' + name # Set customised output directory
      if not os.path.exists(folder_name):
        os.mkdir(folder_name)
        with open(file_path, "rb") as pdf_file:
          images = convert_from_bytes(pdf_file.read(), fmt='jpeg',
                              output_folder= folder_name,
                              dpi=300
                              )
        try:
          os.remove(file_path)
        except NotADirectoryError: # If it is already been deleted
          pass
      else:
        pass # Ensure that it will not create the file that is already created
      return folder_name
    except (ValueError,PDFPageCountError):
      error_file.append(file_path)
  elif check_file==False:
    missing_file.append(file_path)

## Image pre-processing

In [ ]:
from IPython.core.display import Image
from pytesseract import Output
from pytesseract import TesseractError
# Image preprocessing
# Define a function to detect and rotate image
def rotate_image(image_path):
  orientation_angle=pytesseract.image_to_osd(image_path, output_type=Output.DICT)['orientation'] # Angle of orientation
  if orientation_angle == 90:
    image_path = cv2.rotate(image_path, cv2.ROTATE_90_COUNTERCLOCKWISE)
  elif orientation_angle == 180:
    image_path = cv2.rotate(image_path, cv2.ROTATE_180)
  elif orientation_angle == 270:
    image_path = cv2.rotate(image_path, cv2.ROTATE_90_CLOCKWISE)
  else:
    pass
  return image_path

# Preprocessing before OCR to enhance results
def image_preprocessing(lease_folder):
  lease_folder = f"{lease_folder}"
  orig_leases = os.listdir(lease_folder)
  orig_leases = [i for i in orig_leases if '.pdf' in i] # Exclude the non-lease folder
  for leases in range(0,len(orig_leases)):
    orig_leases[leases] = lease_folder + '/' + orig_leases[leases] # Extract all folders from the root folder
  for index in range(0,len(orig_leases)):
    folders = orig_leases[index]
    files = os.listdir(folders) # List all file names in the input folder
    for i in range(0,len(files)):
      files[i] = folders + '/' + files[i] # Above steps to extract all file directories in this directory
    for image in files:
      if image.endswith('.jpg'):
        try:
          image_path = image # Image directory
          img = cv2.imread(image_path , cv2.IMREAD_GRAYSCALE) # Convert image to black and white
          img = rotate_image(img) # Rotate image if is not scanned vertically
          ret3,th3 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) # Binarization
          cv2.imwrite(image_path, th3) #Overwrite original images
        except TesseractError:
          pass
      else:
        pass
    index+=1

##  Combination of above functions

In [ ]:
# Build a pipeline that export pdf to images and perform image preprocessing
def pdf_to_image_process(filepath):
  try:
    filepath = f"{filepath}"
      # Process image
    img_folder_dir = pdf_to_image(filepath)
    image_preprocessing(img_folder_dir)
  except FileNotFoundError:
    pass

In [ ]:
# Process all files in the directory
for i in image_pdf:
  pdf_to_image_process(i)

# Convert image to text

In [ ]:
# Get the folder directories of all folders containing preprocessed images
import os
dir = '/content/drive/MyDrive/Processed_pdf'
files = os.listdir(dir)
for i in range(0,len(files)):
  files[i] = dir + '/' + files[i]

In [ ]:
# Get the file directories of all files in each folder
List_leases = []
for folders in range(0,len(files)):
  leases = os.listdir(files[folders])
  for i in range(0,len(leases)):
    if (('chopped' in leases[i]) or ('.jpg' not in leases[i])):
      leases.remove(leases[i])
    else:
      leases[i] = files[folders] + '/' + leases[i]
      i+=1
  List_leases.append(leases)
  folders +=1

In [ ]:
# Functions to convert image to text
def image_to_text(dir):
  # Get the folder directories of all folders containing preprocessed images
  folders = os.listdir(dir)
  for i in range(0,len(folders)):
    folders[i] = dir + '/' + folders[i]
  # Get the file directories of all files in each folder
  List_leases = []
  for folder in range(0,len(folders)):
    leases = os.listdir(folders[folder])
    for i in range(0,len(leases)):
      if (('chopped' in leases[i]) or ('.jpg' not in leases[i])):
        leases.remove(leases[i])
      else:
        leases[i] = folders[folder] + '/' + leases[i]
        i+=1
    List_leases.append(leases)
    folder +=1
  # Image to text
  error_image = []
  for i in range(len(List_leases)):
      text_scanned=''
      pdf_name= folders[i].split('/')[-1]
      ouput_folder = '/content/drive/MyDrive/Pdf_text'
      output_path = ouput_folder + '/' + pdf_name + '.txt'
      # Check if output path already exists to avoid unnecessary conversion
      if os.path.exists(output_path):
        pass
      else:
        try:
          for x in range(len(List_leases[i])):
            img = cv2.imread(List_leases[i][x])
            text = pytesseract.image_to_string(img)
            text_scanned += text
            #os.remove(dir[i])
            x+=1
          fileTxt = open(output_path, "w")
          fileTxt.write(text_scanned)
        except TypeError:
          error_image.append(List_leases[i])
      i+=1

In [ ]:
# Process all exported pdf files
image_to_text('/content/drive/MyDrive/Processed_pdf')

# Convert text pdf to txt file

In [ ]:
! pip install PyPDF2
import PyPDF2

In [ ]:
! pip install tika

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32625 sha256=5b137d7dbee430367fe936a1beab1e590f5ed5f65a5c6f5fe4d3e837b04bf2ea
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


In [ ]:
# import parser object from tike
from tika import parser

In [ ]:
def text_pdf_to_txt(input_dir_list):
  output_dir = '/content/drive/MyDrive/Pdf_text'
  for i in range(len(input_dir_list)):
    parsed_pdf = parser.from_file(input_dir_list[i])  # opening pdf file
    # parsed_pdf['content'] returns string
    data = parsed_pdf['content']
    # Write to text files
    file_name = input_dir_list[i].split('/')[-1][:-4]
    output_name = output_dir + '/'+ file_name + '.txt'
    with open(output_name,"w+") as f:
      f.writelines(data)
    i+=1

In [ ]:
# Convert text-based pdf to text
text_pdf_to_txt(text_pdf)

# Verification that every pdf is converted to text

In [ ]:
dir_text = '/content/drive/MyDrive/Pdf_text'
converted_pdf = os.listdir(dir)
len(set(converted_pdf))

387

# Image extraction date

## Find all image path contains date

In [99]:
# import libraries
import torch
from functools import partial
from PIL import Image
from transformers import Pix2StructForConditionalGeneration as psg
from transformers import Pix2StructProcessor as psp
import glob

In [ ]:
# Get the folders in the root directory
rootdir = '/content/drive/MyDrive/Processed_pdf'
folderdir = []
for path in glob.glob(f'{rootdir}/*'):
    folderdir.append(path)

In [ ]:
# Define a function to extract values from lease
def extract_and_crop(folder):
  folderdir = folder
  file_image_dir = []
  # Get files in the folder
  for fol in range(len(folderdir)):
    curr_folder = folderdir[fol]
    file_folder = []
    for files in glob.glob(f'{curr_folder}/*'):
      file_folder.append(files)
    # If a lease is less than 3 pages ignore
    if len(file_folder) <=3:
      fol+=1
    else:
  # Search date in the file
      for f in range(0,3): # We only want to extract date from the first 3 pages
        date_pattern = ['date','dated']
        img = cv2.imread(file_folder[f])
        d = pytesseract.image_to_data(img, output_type=Output.DICT) # Get attributes/metadata of the image
        text = d['text']
        text = [i.lower() for i in text]
        response= []
        for i in text:
          res = any(item==i for item in date_pattern)
          response.append(res)
        if True in response:
          file_image_dir.append(file_folder[f])
        else:
          f+=1
    fol+=1
  return file_image_dir

In [ ]:
# Find all images containing 'date' related' text
file_image_dir = extract_and_crop(folderdir)

In [ ]:
# Define a function to remove duplicated pages from the same lease
def remove_duplicate_dir(list_dir):
  duplicate_index= []
  for i in range(len(list_dir)-1):
    folder_name = list_dir[i].split('/')[-2]
    next_folder_name = list_dir[i+1].split('/')[-2]
    if folder_name == next_folder_name:
      duplicate_index.append(i+1)
    i+=1
  # Sort index from largest to smallest
  d = sorted(duplicate_index,reverse=True)
  return d

In [ ]:
# List index of duplicated entry
index_dup = remove_duplicate_dir(file_image_dir)
# Remove duplicate from list by index
for i in range(len(index_dup)):
  del file_image_dir[index_dup[i]]
  i+=1

## Save filedirectory in a pickle file

In [ ]:
# Python program to store list to file using pickle module
import pickle
# write list to binary file
def write_list(a_list):
    # store list in binary file so 'wb' mode
    with open('/content/drive/MyDrive/image_dir', 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')
write_list(file_image_dir)

Done writing list into a binary file


## Read image date

In [ ]:
# Reload back the list
import pickle
with open('/content/drive/MyDrive/image_dir', 'rb') as f:
  image_dir = pickle.load(f)

In [ ]:
index_dup = remove_duplicate_dir(image_dir)
# Remove duplicate from list by index
for i in range(len(index_dup)):
  del image_dir[index_dup[i]]
  i+=1

In [6]:
# Import pix2struct model and processor
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = psg.from_pretrained("google/pix2struct-docvqa-base").to(DEVICE)
processor = psp.from_pretrained("google/pix2struct-docvqa-base")

In [21]:
# Define a function to generate responses
def generate(model, processor, img, questions):
  inputs = processor(images=[img for _ in range(len(questions))],
           text=questions, return_tensors="pt").to(DEVICE)
  predictions = model.generate(**inputs, max_new_tokens=256)
  return zip(questions, processor.batch_decode(predictions, skip_special_tokens=True))

In [ ]:
# Read image
# Create an empty pd dataframe to store data
DOL = pd.DataFrame(columns=['Lease_name','Date of Lease'])
#Define a function to read_image using pix2struct
def read_image(image_path):
  image = Image.open(image_path).convert('RGB')
  question = 'What is the date'
  inputs = processor(images=image, text=question, return_tensors="pt").to("cuda")
  predictions = model.generate(**inputs)
  date = processor.decode(predictions[0], skip_special_tokens=True)
  folder_name = image_path.split('/')[-2]
  lease_date_row = {'Lease_name': folder_name, 'Date of Lease': date}
  DOL.loc[len(DOL)] = lease_date_row

In [ ]:
for i in range(len(image_dir)):
  read_image(image_dir[i])
  i+=1

In [ ]:
# Show all rows #Large
pd.set_option('display.max_rows', None)
DOL

,Lease_name,Date of Lease
0,Lease Dated 06.03.2007,6th March 2007
1,CVS 113 - Section 25 Notice - Lease Renewal No...,17 March 2022
2,CVS 113 - Section 25 Notice - Lease Renewal No...,17 March 2022
3,((CVS380) Completed Lease - 24 St Margaret Str...,4 December 2015
4,(CVS366) Frome - Lease dated 10 Oct 2014,10th October 2014
5,Lease Renewal (signed by CVS) dated 14.03.23,14 march
6,Lease Renewal (signed by CVS) dated 14.03.23,14 march 2022
7,CVS032 - Linden House - Lease dated 28.02.2020,1979
8,Landlord Signed Lease 28.10.16 (2),28th October 2016
9,CVS028 Coastline - 1 Village Rise Lowestoft Le...,8 October 2021


In [ ]:
# Save to file in parquet and csv
DOL.to_parquet('DOL.parquet')
DOL.to_csv('DOL.csv')

# Upload transformed text file to sharepoint

In [ ]:
sharepoint_url = "https://davittjonesbouldltd.sharepoint.com/sites/Files"
sharepoint_files = 'Shared Documents'
site_name = 'Files'
username = 'difan.deng@djblaw.co.uk'
password = 'PaperMoon543!'
folder_name = 'Adminwork/Difan Deng/Lease'
text_folder_name = 'Adminwork/Difan Deng/Lease_text'

In [ ]:
# Define functions to upload a single file
from concurrent.futures import ThreadPoolExecutor, as_completed
def upload_to_sharepoint(connection,files, folder,filefolder):
    target_folder_path = f'{files}/{folder}'
    target_folder = connection.web.get_folder_by_server_relative_url(target_folder_path)
    file_list = os.listdir(filefolder)
    for i in range(len(file_list)):
      file_name = filefolder + '/' + file_list[i]
      with open(f'{file_name}', 'rb') as content_file:
        file_content = content_file.read()
        target_folder.upload_file(file_name.split('/')[-1], file_content).execute_query()
      i+=1

In [ ]:
# Upload to sharepoint
file_directory = '/content/drive/MyDrive/Pdf_text' # Customised for your own folder
upload_to_sharepoint(connection_sharepoint,sharepoint_files,text_folder_name,file_directory)